# Invoke and Create Skillset using Azure Search

Notebook walking through the process of setting up a skillset on an azure search service

## Import Dependencies

In [1]:
import requests
import json
import sys
import configparser

sys.path.append("../")
from ml.models.search_service import SearchService

## Configure Variables 

In [2]:
# Configure all required variables for Cognitive Search. Replace each with the credentials from your accounts.
# The required services are created as part of the initial resource deployment step

## Configure variables for this example
config = configparser.ConfigParser()
config.read('config.ini')
config_parameters = config['DEFAULT']

# Replace with Search Service name, API key, and endpoint from the Azure portal.
search_service_name = config_parameters["search_service_name"]
api_key = config_parameters["api_key"]
search_service = config_parameters["search_service"]
# Leave the API version and content_type as they are listed here.
api_version = config_parameters["api_version"]
content_type = 'application/json'

# Replace with a cognitive services key.
cog_svcs_key = config_parameters["cog_svcs_key"]
cog_svcs_acct = config_parameters["cog_svcs_acct"]

#Name of the storage account. This will be used for the datasource, knowledge store
STORAGEACCOUNTNAME = config_parameters["STORAGEACCOUNTNAME"]
STORAGEACCOUNTKEY = config_parameters["STORAGEACCOUNTKEY"]
STORAGECONNSTRING = config_parameters["STORAGECONNSTRING"]

# Create a container and specify that below
# This sample assumes you will use the same storage account for the datasource, knowledge store and indexer cache
datasource_container = 'new-test-subset'
indexername = f'{datasource_container}-idxr'

know_store_cache = config_parameters["know_store_cache"]

# Resource group your AML inference cluster will be deployed in 
resource_group = config_parameters["resource_group"]
subscription_id = config_parameters["subscription_id"]
workspace_name = config_parameters["workspace_name"]

## Instanstiate Azure Search Object

In [3]:
search_object = SearchService(api_key=api_key,
                               endpoint=search_service,
                               cog_svcs_key=cog_svcs_key)

## Create Data Source 

In [4]:
search_object.create_data_source(datasource_container=datasource_container,
                                  storage_connection_string=STORAGECONNSTRING,
                                  description="test datasource for oneweek")

<Response [201]>
{
  "@odata.context": "https://oneweek-search-service.search.windows.net/$metadata#datasources/$entity",
  "@odata.etag": "\"0x8D8809802471B99\"",
  "name": "new-test-subset-ds",
  "description": "test datasource for oneweek",
  "type": "azureblob",
  "subtype": null,
  "credentials": {
    "connectionString": null
  },
  "container": {
    "name": "new-test-subset",
    "query": null
  },
  "dataChangeDetectionPolicy": null,
  "dataDeletionDetectionPolicy": null,
  "encryptionKey": null
}


{'@odata.context': 'https://oneweek-search-service.search.windows.net/$metadata#datasources/$entity',
 '@odata.etag': '"0x8D8809802471B99"',
 'name': 'new-test-subset-ds',
 'description': 'test datasource for oneweek',
 'type': 'azureblob',
 'subtype': None,
 'credentials': {'connectionString': None},
 'container': {'name': 'new-test-subset', 'query': None},
 'dataChangeDetectionPolicy': None,
 'dataDeletionDetectionPolicy': None,
 'encryptionKey': None}

## Create Skillset

In [5]:
skills = [
        {
            '@odata.type': '#Microsoft.Skills.Text.KeyPhraseExtractionSkill',
            'name': '#1',
            'description': None,
            'context': '/document/reviews_text',
            'defaultLanguageCode': 'en',
            'maxKeyPhraseCount': None,
            'inputs': [
                {
                    'name': 'text',
                    'source': '/document/reviews_text',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'languageCode',
                    'source': '/document/language',
                    'sourceContext': None,
                    'inputs': []
                }
            ],
            'outputs': [
                {
                    'name': 'keyPhrases',
                    'targetName': 'keyphrases'
                }
            ]
        },
        {
            '@odata.type': '#Microsoft.Skills.Text.LanguageDetectionSkill',
            'name': '#2',
            'description': None,
            'context': '/document',
            'inputs': [
                {
                    'name': 'text',
                    'source': '/document/reviews_text',
                    'sourceContext': None,
                    'inputs': []
                }
            ],
            'outputs': [
                {
                    'name': 'languageCode',
                    'targetName': 'language'
                }
            ]
        },
        {'@odata.type': '#Microsoft.Skills.Text.TranslationSkill',
            'name': '#3',
            'description': None,
            'context': '/document/reviews_text',
            'defaultFromLanguageCode': None,
            'defaultToLanguageCode': 'en',
            'suggestedFrom': 'en',
            'inputs': [
                {
                    'name': 'text',
                    'source': '/document/reviews_text',
                    'sourceContext': None,
                    'inputs': []
                }
            ],
            'outputs': [
                {
                    'name': 'translatedText',
                    'targetName': 'translated_text'
                }
            ]
        },
        {
            '@odata.type': '#Microsoft.Skills.Util.ShaperSkill',
            'name': '#4',
            'description': None,
            'context': '/document',
            'inputs': [
                {
                    'name': 'address',
                    'source': '/document/address',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'categories',
                    'source': '/document/categories',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'city',
                    'source': '/document/city',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'country',
                    'source': '/document/country',
                    'sourceContext': None,
                    'inputs': []
                },
                {'name': 'latitude',
                    'source': '/document/latitude',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'longitude',
                    'source': '/document/longitude',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'name',
                    'source': '/document/name',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'postalCode',
                    'source': '/document/postalCode',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'province',
                    'source': '/document/province',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'reviews_date',
                    'source': '/document/reviews_date',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'reviews_dateAdded',
                    'source': '/document/reviews_dateAdded',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'reviews_rating',
                    'source': '/document/reviews_rating',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'reviews_text',
                    'source': '/document/reviews_text',
                    'sourceContext': None,
                    'inputs': []
                },
                { 'name': 'reviews_title',
                    'source': '/document/reviews_title',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'reviews_username',
                    'source': '/document/reviews_username',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'AzureSearch_DocumentKey',
                    'source': '/document/AzureSearch_DocumentKey',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_content_type',
                    'source': '/document/metadata_storage_content_type',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_size',
                    'source': '/document/metadata_storage_size',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_last_modified',
                    'source': '/document/metadata_storage_last_modified',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_content_md5',
                    'source': '/document/metadata_storage_content_md5',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_name',
                    'source': '/document/metadata_storage_name',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_path',
                    'source': '/document/metadata_storage_path',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'metadata_storage_file_extension',
                    'source': '/document/metadata_storage_file_extension',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'keyPhrases',
                    'source': '/document/reviews_text/keyphrases/*',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'languageCode',
                    'source': '/document/language',
                    'sourceContext': None,
                    'inputs': []
                },
                {
                    'name': 'translatedText',
                    'source': '/document/reviews_text/translated_text',
                    'sourceContext': None,
                    'inputs': []
                }
            ],
            'outputs': [
                {
                    'name': 'output',
                    'targetName': 'objectprojection'
                }
            ]
        }
    ]

In [6]:
knowledge_store = {
        'storageConnectionString': f'{STORAGECONNSTRING}',
        'projections': [
            {
                'tables': [],
                'objects': [{
                        'storageContainer':  f'{datasource_container}-enriched',
                        'referenceKeyName': None,
                        'generatedKeyName': None,
                        'source': '/document/objectprojection',
                        'sourceContext': None,
                        'inputs': []
                    }
                ],
                'files': []
            }
        ]
    }

In [7]:
search_object.create_skillset(datasource_container=datasource_container,
                               skills=skills,
                               knowledge_store=knowledge_store,
                               description="testing create skillset function")

<Response [201]>
{
  "@odata.context": "https://oneweek-search-service.search.windows.net/$metadata#skillsets/$entity",
  "@odata.etag": "\"0x8D880980282100C\"",
  "name": "new-test-subset-ss",
  "description": "testing create skillset function",
  "skills": [
    {
      "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
      "name": "#1",
      "description": null,
      "context": "/document/reviews_text",
      "defaultLanguageCode": "en",
      "maxKeyPhraseCount": null,
      "inputs": [
        {
          "name": "text",
          "source": "/document/reviews_text",
          "sourceContext": null,
          "inputs": []
        },
        {
          "name": "languageCode",
          "source": "/document/language",
          "sourceContext": null,
          "inputs": []
        }
      ],
      "outputs": [
        {
          "name": "keyPhrases",
          "targetName": "keyphrases"
        }
      ]
    },
    {
      "@odata.type": "#Microsoft.Skills.Text.L

{'@odata.context': 'https://oneweek-search-service.search.windows.net/$metadata#skillsets/$entity',
 '@odata.etag': '"0x8D880980282100C"',
 'name': 'new-test-subset-ss',
 'description': 'testing create skillset function',
 'skills': [{'@odata.type': '#Microsoft.Skills.Text.KeyPhraseExtractionSkill',
   'name': '#1',
   'description': None,
   'context': '/document/reviews_text',
   'defaultLanguageCode': 'en',
   'maxKeyPhraseCount': None,
   'inputs': [{'name': 'text',
     'source': '/document/reviews_text',
     'sourceContext': None,
     'inputs': []},
    {'name': 'languageCode',
     'source': '/document/language',
     'sourceContext': None,
     'inputs': []}],
   'outputs': [{'name': 'keyPhrases', 'targetName': 'keyphrases'}]},
  {'@odata.type': '#Microsoft.Skills.Text.LanguageDetectionSkill',
   'name': '#2',
   'description': None,
   'context': '/document',
   'inputs': [{'name': 'text',
     'source': '/document/reviews_text',
     'sourceContext': None,
     'inputs': []

## Create Index

In [8]:
fields =[
        {
            "name": "address",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "categories",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "city",
            "type": "Edm.String",
            "searchable": False,
            "filterable": True,
            "retrievable": True,
            "sortable": False,
            "facetable": True,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "country",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": True,
            "key": False,
            "indexAnalyzer": None,"searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "latitude",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "longitude",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "name",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": True,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "postalCode",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": True,
            "key": False,"indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "province",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "reviews_date",
            "type": "Edm.DateTimeOffset",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "reviews_dateAdded",
            "type": "Edm.DateTimeOffset",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "reviews_rating",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,"indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "reviews_text",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "en.microsoft",
            "synonymMaps": []
        },
        {
            "name": "reviews_title",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "en.microsoft",
            "synonymMaps": []
        },
        {
            "name": "reviews_username",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "AzureSearch_DocumentKey",
            "type": "Edm.String",
            "searchable": False,"filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": True,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_content_type",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_size",
            "type": "Edm.Int64",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_last_modified",
            "type": "Edm.DateTimeOffset",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_content_md5",
            "type": "Edm.String","searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_name",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_path",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_file_extension",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "keyphrases","type": "Collection(Edm.String)",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "en.microsoft",
            "synonymMaps": []
        },
        {
            "name": "language",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "en.microsoft",
            "synonymMaps": []
        },
        {
            "name": "translated_text",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "en.lucene",
            "synonymMaps": []
        },
        {
            "name": "aspect_sentiment",
            "type": "Collection(Edm.ComplexType)",
            "fields": [
                {
                    "name": "text",
                    "type": "Edm.String",
                    "searchable": True,
                    "filterable": False,
                    "retrievable": True,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                    "indexAnalyzer": None,"searchAnalyzer": None,
                    "analyzer": "standard.lucene",
                    "synonymMaps": []
                },
                {
                    "name": "type",
                    "type": "Edm.String",
                    "searchable": False,
                    "filterable": True,
                    "retrievable": True,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                    "indexAnalyzer": None,
                    "searchAnalyzer": None,
                    "analyzer": None,
                    "synonymMaps": []
                },
                {
                    "name": "polarity",
                    "type": "Edm.String",
                    "searchable": False,
                    "filterable": True,
                    "retrievable": True,
                    "sortable": False,
                    "facetable": True,
                    "key": False,
                    "indexAnalyzer": None,
                    "searchAnalyzer": None,
                    "analyzer": None,
                    "synonymMaps": []
                },
                {
                    "name": "score",
                    "type": "Edm.Double",
                    "searchable": False,
                    "filterable": False,
                    "retrievable": True,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                    "indexAnalyzer": None,
                    "searchAnalyzer": None,
                    "analyzer": None,
                    "synonymMaps": []
                },
                {
                    "name": "start",
                    "type": "Edm.Int32",
                    "searchable": False,
                    "filterable": False,
                    "retrievable": True,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                    "indexAnalyzer": None,"searchAnalyzer": None,
                    "analyzer": None,
                    "synonymMaps": []
                },
                {
                    "name": "len",
                    "type": "Edm.Int32",
                    "searchable": False,
                    "filterable": False,
                    "retrievable": True,
                    "sortable": False,
                    "facetable": False,
                    "key": False,
                    "indexAnalyzer": None,
                    "searchAnalyzer": None,
                    "analyzer": None,
                    "synonymMaps": []
                }
            ]
        }
    ]

In [9]:
suggesters=[
        {
            "name": "sg",
            "searchMode": "analyzingInfixMatching",
            "sourceFields": [
                "reviews_text"
            ]
        }
    ]

In [10]:
search_object.create_index(datasource_container=datasource_container,
                            fields = fields,
                            suggesters =suggesters)

<Response [201]>
{
  "@odata.context": "https://oneweek-search-service.search.windows.net/$metadata#indexes/$entity",
  "@odata.etag": "\"0x8D8809803123E00\"",
  "name": "new-test-subset-idx",
  "defaultScoringProfile": "",
  "fields": [
    {
      "name": "address",
      "type": "Edm.String",
      "searchable": false,
      "filterable": false,
      "retrievable": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "indexAnalyzer": null,
      "searchAnalyzer": null,
      "analyzer": null,
      "synonymMaps": []
    },
    {
      "name": "categories",
      "type": "Edm.String",
      "searchable": false,
      "filterable": false,
      "retrievable": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "indexAnalyzer": null,
      "searchAnalyzer": null,
      "analyzer": null,
      "synonymMaps": []
    },
    {
      "name": "city",
      "type": "Edm.String",
      "searchable": false,
      "filterable": true,


{'@odata.context': 'https://oneweek-search-service.search.windows.net/$metadata#indexes/$entity',
 '@odata.etag': '"0x8D8809803123E00"',
 'name': 'new-test-subset-idx',
 'defaultScoringProfile': '',
 'fields': [{'name': 'address',
   'type': 'Edm.String',
   'searchable': False,
   'filterable': False,
   'retrievable': True,
   'sortable': False,
   'facetable': False,
   'key': False,
   'indexAnalyzer': None,
   'searchAnalyzer': None,
   'analyzer': None,
   'synonymMaps': []},
  {'name': 'categories',
   'type': 'Edm.String',
   'searchable': False,
   'filterable': False,
   'retrievable': True,
   'sortable': False,
   'facetable': False,
   'key': False,
   'indexAnalyzer': None,
   'searchAnalyzer': None,
   'analyzer': None,
   'synonymMaps': []},
  {'name': 'city',
   'type': 'Edm.String',
   'searchable': False,
   'filterable': True,
   'retrievable': True,
   'sortable': False,
   'facetable': True,
   'key': False,
   'indexAnalyzer': None,
   'searchAnalyzer': None,
   

## Create Indexer

In [11]:
parameters = {
        "batchSize": None,
        "maxFailedItems": 0,
        "maxFailedItemsPerBatch": 0,
        "base64EncodeKeys": None,
        "configuration": {
            "dataToExtract": "contentAndMetadata",
            "parsingMode": "delimitedText",
            "firstLineContainsHeaders": True,
            "delimitedTextDelimiter": ",",
            "delimitedTextHeaders": ""
        }
    }

In [12]:
field_mappings = [
        {
            "sourceFieldName": "AzureSearch_DocumentKey",
            "targetFieldName": "AzureSearch_DocumentKey",
            "mappingFunction": {
                "name": "base64Encode",
                "parameters": None
            }
        }
    ]

In [13]:
output_field_mappings = [
        {
            "sourceFieldName": "/document/reviews_text/keyphrases",
            "targetFieldName": "keyphrases",
            "mappingFunction": None
        },
        {
            "sourceFieldName": "/document/language",
            "targetFieldName": "language",
            "mappingFunction": None
        },
        {
            "sourceFieldName": "/document/reviews_text/translated_text",
            "targetFieldName": "translated_text",
            "mappingFunction": None
        }
    ]

In [14]:
search_object.create_indexer(datasource_container=datasource_container,
                              description="testing create indexer function",
                              parameters=parameters,
                              field_mappings=field_mappings,
                              output_field_mappings=output_field_mappings,
                              know_store_cache=know_store_cache)

<Response [201]>
{
  "@odata.context": "https://oneweek-search-service.search.windows.net/$metadata#indexers/$entity",
  "@odata.etag": "\"0x8D88098037BA0B1\"",
  "name": "new-test-subset-idxr",
  "description": "testing create indexer function",
  "dataSourceName": "new-test-subset-ds",
  "skillsetName": "new-test-subset-ss",
  "targetIndexName": "new-test-subset-idx",
  "disabled": null,
  "schedule": {
    "interval": "PT2H",
    "startTime": "0001-01-01T00:00:00Z"
  },
  "parameters": {
    "batchSize": null,
    "maxFailedItems": 0,
    "maxFailedItemsPerBatch": 0,
    "base64EncodeKeys": null,
    "configuration": {
      "dataToExtract": "contentAndMetadata",
      "parsingMode": "delimitedText",
      "firstLineContainsHeaders": true,
      "delimitedTextDelimiter": ",",
      "delimitedTextHeaders": ""
    }
  },
  "fieldMappings": [
    {
      "sourceFieldName": "AzureSearch_DocumentKey",
      "targetFieldName": "AzureSearch_DocumentKey",
      "mappingFunction": {
        

{'@odata.context': 'https://oneweek-search-service.search.windows.net/$metadata#indexers/$entity',
 '@odata.etag': '"0x8D88098037BA0B1"',
 'name': 'new-test-subset-idxr',
 'description': 'testing create indexer function',
 'dataSourceName': 'new-test-subset-ds',
 'skillsetName': 'new-test-subset-ss',
 'targetIndexName': 'new-test-subset-idx',
 'disabled': None,
 'schedule': {'interval': 'PT2H', 'startTime': '0001-01-01T00:00:00Z'},
 'parameters': {'batchSize': None,
  'maxFailedItems': 0,
  'maxFailedItemsPerBatch': 0,
  'base64EncodeKeys': None,
  'configuration': {'dataToExtract': 'contentAndMetadata',
   'parsingMode': 'delimitedText',
   'firstLineContainsHeaders': True,
   'delimitedTextDelimiter': ',',
   'delimitedTextHeaders': ''}},
 'fieldMappings': [{'sourceFieldName': 'AzureSearch_DocumentKey',
   'targetFieldName': 'AzureSearch_DocumentKey',
   'mappingFunction': {'name': 'base64Encode', 'parameters': None}}],
 'outputFieldMappings': [{'sourceFieldName': '/document/reviews_

## Get Indexer Status

In [17]:
search_object.get_indexer_status(indexer_name= indexername)

<Response [200]>
{
  "@odata.context": "https://oneweek-search-service.search.windows.net/$metadata#Microsoft.Azure.Search.V2019_05_06_Preview.IndexerExecutionInfo",
  "name": "new-test-subset-idxr",
  "status": "running",
  "lastResult": {
    "status": "inProgress",
    "errorMessage": null,
    "startTime": "2020-11-04T08:02:47.02Z",
    "endTime": null,
    "itemsProcessed": 0,
    "itemsFailed": 0,
    "initialTrackingState": null,
    "finalTrackingState": null,
    "mode": "indexingAllDocs",
    "errors": [],
    "warnings": [],
    "metrics": null
  },
  "executionHistory": [],
  "limits": {
    "maxRunTime": "PT2H",
    "maxDocumentExtractionSize": 134217728,
    "maxDocumentContentCharactersToExtract": 4194304
  },
  "currentState": {
    "mode": "indexingAllDocs",
    "allDocsInitialTrackingState": null,
    "allDocsFinalTrackingState": null,
    "resetDocsInitialTrackingState": null,
    "resetDocsFinalTrackingState": null,
    "resetDocumentKeys": []
  }
}


## Run Indexer 

In [18]:
search_object.run_indexer(indexer_name=indexername)

<Response [409]>
{
  "error": {
    "code": "",
    "message": "Another indexer invocation is currently in progress; concurrent invocations are not allowed."
  }
}


{'error': {'code': '',
  'message': 'Another indexer invocation is currently in progress; concurrent invocations are not allowed.'}}

## Update Indexer

In [ ]:
search_object.update_indexer(datasource_container=datasource_container,
                              description="testing create indexer function",
                              parameters=parameters,
                              field_mappings=field_mappings,
                              output_field_mappings=output_field_mappings,
                              know_store_cache=know_store_cache)